In [1]:
!pip install matplotlib torch_geometric torch gdown --quiet

In [ ]:
!git clone --branch baselineCe https://github.com/Graph-Classification-Noisy-Label/hackaton.git

In [2]:
%cd hackaton/

/home/onyxia/work/DL-Hackathon/hackaton


In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1Z-1JkPJ6q4C6jX4brvq1VRbJH5RPUCAk -O datasets


In [3]:
!ls -lh datasets

total 16K
drwxr-sr-x 2 onyxia users 4.0K May 26 16:09 A
drwxr-sr-x 2 onyxia users 4.0K May 26 16:09 B
drwxr-sr-x 2 onyxia users 4.0K May 26 16:10 C
drwxr-sr-x 2 onyxia users 4.0K May 26 16:10 D


In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
import logging
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from torch.utils.data import random_split
# Load utility functions from cloned repository
from src.loadData import GraphDataset
from src.utils import set_seed
from src.models import GNN
import argparse

# Set the random seed
set_seed()


os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [5]:
# Modification de GraphDataset pour récupérer les indices (ajout de get)

import gzip
import json
import torch
from torch_geometric.data import Dataset, Data
from torch_geometric.loader import DataLoader

class GraphDataset(Dataset):
    def __init__(self, filename, transform=None, pre_transform=None):
        self.raw = filename
        self.num_graphs, self.graphs_dicts = self._count_graphs() 
        super().__init__(None, transform, pre_transform)

    def len(self):
        return self.num_graphs  
    
    def get(self, idx):
        data = dictToGraphObject(self.graphs_dicts[idx])
        data.idx = idx  # 🔥 Ajoute l'indice global dans l'objet Data
        return data

    def _count_graphs(self):
        with gzip.open(self.raw, "rt", encoding="utf-8") as f:
            graphs_dicts = json.load(f)
            return len(graphs_dicts), graphs_dicts

def dictToGraphObject(graph_dict):
    edge_index = torch.tensor(graph_dict["edge_index"], dtype=torch.long)
    edge_attr = torch.tensor(graph_dict["edge_attr"], dtype=torch.float) if graph_dict["edge_attr"] else None
    num_nodes = graph_dict["num_nodes"]
    y = torch.tensor(graph_dict["y"][0], dtype=torch.long) if graph_dict["y"] is not None else None
    return Data(edge_index=edge_index, edge_attr=edge_attr, num_nodes=num_nodes, y=y)

In [6]:
def add_zeros(data):
    data.x = torch.zeros(data.num_nodes, dtype=torch.long)
    return data

In [7]:
def evaluate(data_loader, model, device, calculate_accuracy=False):
    model.eval()
    correct = 0
    total = 0
    predictions = []
    total_loss = 0
    criterion = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        for data in tqdm(data_loader, desc="Iterating eval graphs", unit="batch"):
            data = data.to(device)
            output = model(data)
            pred = output.argmax(dim=1)
            
            if calculate_accuracy:
                correct += (pred == data.y).sum().item()
                total += data.y.size(0)
                total_loss += criterion(output, data.y).item()
            else:
                predictions.extend(pred.cpu().numpy())
    if calculate_accuracy:
        accuracy = correct / total
        return  total_loss / len(data_loader),accuracy
    return predictions

In [8]:
def save_predictions(predictions, test_path):
    script_dir = os.getcwd() 
    submission_folder = os.path.join(script_dir, "submission")
    test_dir_name = os.path.basename(os.path.dirname(test_path))
    
    os.makedirs(submission_folder, exist_ok=True)
    
    output_csv_path = os.path.join(submission_folder, f"testset_{test_dir_name}.csv")
    
    test_graph_ids = list(range(len(predictions)))
    output_df = pd.DataFrame({
        "id": test_graph_ids,
        "pred": predictions
    })
    
    output_df.to_csv(output_csv_path, index=False)
    print(f"Predictions saved to {output_csv_path}")

In [9]:
def plot_training_progress(train_losses, train_accuracies, output_dir):
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 6))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label="Training Loss", color='blue')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Loss per Epoch')

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label="Training Accuracy", color='green')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Training Accuracy per Epoch')

    # Save plots in the current directory
    os.makedirs(output_dir, exist_ok=True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "training_progress.png"))
    plt.close()

In [10]:
def get_user_input(prompt, default=None, required=False, type_cast=str):

    while True:
        user_input = input(f"{prompt} [{default}]: ")
        
        if user_input == "" and required:
            print("This field is required. Please enter a value.")
            continue
        
        if user_input == "" and default is not None:
            return default
        
        if user_input == "" and not required:
            return None
        
        try:
            return type_cast(user_input)
        except ValueError:
            print(f"Invalid input. Please enter a valid {type_cast.__name__}.")

In [11]:
def get_arguments():
    args = {}
    args['train_path'] = get_user_input("Path to the training dataset (optional)")
    args['test_path'] = get_user_input("Path to the test dataset", required=True)
    args['num_checkpoints'] = get_user_input("Number of checkpoints to save during training", type_cast=int)
    args['device'] = get_user_input("Which GPU to use if any", default=1, type_cast=int)
    args['gnn'] = get_user_input("GNN type (gin, gin-virtual, gcn, gcn-virtual)", default='gin')
    args['drop_ratio'] = get_user_input("Dropout ratio", default=0.0, type_cast=float)
    args['num_layer'] = get_user_input("Number of GNN message passing layers", default=5, type_cast=int)
    args['emb_dim'] = get_user_input("Dimensionality of hidden units in GNNs", default=300, type_cast=int)
    args['batch_size'] = get_user_input("Input batch size for training", default=32, type_cast=int)
    args['epochs'] = get_user_input("Number of epochs to train", default=10, type_cast=int)
    args['baseline_mode'] = get_user_input("Baseline mode: 1 (CE), 2 (GCOD)", default=1, type_cast=int)
    args['noise_prob'] = get_user_input("Noise probability p (used if baseline_mode=2)", default=0.2, type_cast=float)

    
    return argparse.Namespace(**args)


In [24]:
def populate_args(args):
    print("Arguments received:")
    for key, value in vars(args).items():
        print(f"{key}: {value}")
args = get_arguments()
populate_args(args)

Path to the training dataset (optional) [None]:  datasets/D/train.json.gz
Path to the test dataset [None]:  datasets/D/test.json.gz
Number of checkpoints to save during training [None]:  50
Which GPU to use if any [1]:  
GNN type (gin, gin-virtual, gcn, gcn-virtual) [gin]:  gcn
Dropout ratio [0.0]:  
Number of GNN message passing layers [5]:  
Dimensionality of hidden units in GNNs [300]:  
Input batch size for training [32]:  
Number of epochs to train [10]:  50
Baseline mode: 1 (CE), 2 (GCOD) [1]:  2
Noise probability p (used if baseline_mode=2) [0.2]:  


Arguments received:
train_path: datasets/D/train.json.gz
test_path: datasets/D/test.json.gz
num_checkpoints: 50
device: 1
gnn: gcn
drop_ratio: 0.0
num_layer: 5
emb_dim: 300
batch_size: 32
epochs: 50
baseline_mode: 2
noise_prob: 0.2


In [25]:
script_dir = os.getcwd() 
# device = torch.device(f"cuda:{args.device}" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_checkpoints = args.num_checkpoints if args.num_checkpoints else 3
    
if args.gnn == 'gin':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gin-virtual':
    model = GNN(gnn_type='gin', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
elif args.gnn == 'gcn':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=False).to(device)
elif args.gnn == 'gcn-virtual':
    model = GNN(gnn_type='gcn', num_class=6, num_layer=args.num_layer, emb_dim=args.emb_dim, drop_ratio=args.drop_ratio, virtual_node=True).to(device)
else:
    raise ValueError('Invalid GNN type')
    
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# criterion = torch.nn.CrossEntropyLoss()
if args.baseline_mode == 2:
    # On initialise tout (criterion et optimizer) dans la boucle après la définition de full_dataset, car on a besoin du nombre de sample
    pass
else:
    criterion = torch.nn.CrossEntropyLoss()

In [26]:
test_dir_name = os.path.basename(os.path.dirname(args.test_path))
logs_folder = os.path.join(script_dir, "logs", test_dir_name)
log_file = os.path.join(logs_folder, "training.log")
os.makedirs(os.path.dirname(log_file), exist_ok=True)
logging.basicConfig(filename=log_file, level=logging.INFO, format='%(asctime)s - %(message)s')
logging.getLogger().addHandler(logging.StreamHandler())

checkpoint_path = os.path.join(script_dir, "checkpoints", f"model_{test_dir_name}_best.pth")
checkpoints_folder = os.path.join(script_dir, "checkpoints", test_dir_name)
os.makedirs(checkpoints_folder, exist_ok=True)


In [27]:
if os.path.exists(checkpoint_path) and not args.train_path:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f"Loaded best model from {checkpoint_path}")

In [28]:
class IndexedSubDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, i):
        data = self.subset[i]
        data.idx = i  # 👈 réindexe localement
        return data

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GCODLoss(nn.Module):
    #def __init__(self, num_samples, num_classes=6, alpha=1.0, beta=1.0):
    def __init__(self, num_classes=6, alpha=1.0, beta=1.0):
        super().__init__()
        self.num_classes = num_classes
        self.alpha = alpha
        self.beta = beta

        # u est un vecteur trainable de taille num_samples
        #self.register_parameter("u", nn.Parameter(torch.ones(num_samples)))

    def forward(self, logits, u_batch, y_onehot, y_soft, a_train):
        """
        logits: (B, C)
        u_batch: (B,)
        y_onehot: (B, C)
        y_soft: (B, C)
        a_train: float
        """
        B, C = logits.shape

        u_diag_y = u_batch.unsqueeze(1) * y_onehot  # (B, C)

        # L1: weighted cross-entropy with soft labels
        logits_mod = logits + a_train * u_diag_y
        L1 = F.cross_entropy(logits_mod, y_soft.argmax(dim=1))

        # L2: squared norm alignment
        y_pred_onehot = torch.zeros_like(y_onehot)
        y_pred_onehot.scatter_(1, logits.argmax(dim=1, keepdim=True), 1)
        L2 = (1 / C) * ((y_pred_onehot + u_diag_y - y_onehot).pow(2).sum(dim=1).mean())

        # L3: KL regularization
        diag_logits_y = (logits * y_onehot).sum(dim=1)
        L = torch.log(torch.sigmoid(diag_logits_y) + 1e-12)
        sigma_neg_log_u = 1.0 / (1.0 + u_batch)

        P = torch.sigmoid(L)
        Q = sigma_neg_log_u.clamp(min=1e-6, max=1 - 1e-6)
        L3 = (1 - a_train) * (P * (torch.log(P + 1e-12) - torch.log(Q))).mean()

        return L1 + L3, L2

In [30]:
def train(train_loader, model, optimizer_theta, optimizer_u, criterion, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in train_loader:
        batch = batch.to(device)

        optimizer_theta.zero_grad()
        optimizer_u.zero_grad()

        logits = model(batch)

        y = batch.y
        y_onehot = F.one_hot(y, num_classes=criterion.num_classes).float()
        y_soft = y_onehot.clone()

        # Récupère les indices des graphes dans le batch
        #graph_indices = batch.idx.to(criterion.u.device)        
        #u_batch = criterion.u[graph_indices]

        # Récupération des indices (par exemple avec batch.ptr)
        graph_indices = batch.idx.to(device)
        u_batch = u[graph_indices]  # 🔁 u est passé de l'extérieur


        # ➕ Calcul des prédictions pour accuracy
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)
        train_acc = correct / total

        # ➕ Calcul des pertes
        loss_theta, loss_u = criterion(logits, u_batch, y_onehot, y_soft, train_acc)

        # ➕ Backpropagation
        loss_theta.backward(retain_graph=True)
        optimizer_theta.step()

        loss_u.backward()
        optimizer_u.step()

        # ✅ Clamp u entre 0 et 1 pour éviter qu’il devienne négatif ou explose
        u.data.clamp_(0, 1)


        total_loss += loss_theta.item()

    avg_loss = total_loss / len(train_loader)
    final_train_acc = correct / total
    return avg_loss, final_train_acc


In [31]:
if args.train_path:
    full_dataset = GraphDataset(args.train_path, transform=add_zeros)
    val_size = int(0.2 * len(full_dataset))
    train_size = len(full_dataset) - val_size

    generator = torch.Generator().manual_seed(12)
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=generator)

    train_dataset = IndexedSubDataset(train_dataset)

    if args.baseline_mode == 2:

        u = nn.Parameter(torch.ones(len(train_dataset), device=device))
        optimizer_u = torch.optim.Adam([u], lr=1)
        optimizer_theta = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
        
        criterion = GCODLoss(num_classes=6, alpha=1.0, beta=1.0)


    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False)  # shuffle=False obligatoire
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False)

    num_epochs = args.epochs
    best_val_accuracy = 0.0

    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    if num_checkpoints > 1:
        checkpoint_intervals = [int((i + 1) * num_epochs / num_checkpoints) for i in range(num_checkpoints)]
    else:
        checkpoint_intervals = [num_epochs]

    for epoch in range(num_epochs):
        train_loss, train_acc = train(
            train_loader, model, optimizer_theta, optimizer_u, criterion, device
        )

        val_loss, val_acc = evaluate(val_loader, model, device, calculate_accuracy=True)

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        logging.info(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        if val_acc > best_val_accuracy:
            best_val_accuracy = val_acc
            torch.save(model.state_dict(), checkpoint_path)
            print(f"Best model updated and saved at {checkpoint_path}")

    plot_training_progress(train_losses, train_accuracies, os.path.join(logs_folder, "plots"))
    plot_training_progress(val_losses, val_accuracies, os.path.join(logs_folder, "plotsVal"))

Iterating eval graphs: 100%|██████████| 65/65 [00:13<00:00,  4.87batch/s]
Epoch 1/50, Loss: 0.9459, Train Acc: 0.5066, Val Acc: 0.2758
Epoch 1/50, Loss: 0.9459, Train Acc: 0.5066, Val Acc: 0.2758


Epoch 1/50, Loss: 0.9459, Train Acc: 0.5066, Val Acc: 0.2758
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:12<00:00,  5.01batch/s]
Epoch 2/50, Loss: 0.6187, Train Acc: 0.6007, Val Acc: 0.3181
Epoch 2/50, Loss: 0.6187, Train Acc: 0.6007, Val Acc: 0.3181


Epoch 2/50, Loss: 0.6187, Train Acc: 0.6007, Val Acc: 0.3181
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:13<00:00,  4.91batch/s]
Epoch 3/50, Loss: 0.4832, Train Acc: 0.6577, Val Acc: 0.6873
Epoch 3/50, Loss: 0.4832, Train Acc: 0.6577, Val Acc: 0.6873


...

Epoch 48/50, Loss: 0.0042, Train Acc: 0.9995, Val Acc: 0.7349
Best model updated and saved at /home/onyxia/work/DL-Hackathon/hackaton/checkpoints/model_D_best.pth


Iterating eval graphs: 100%|██████████| 65/65 [00:12<00:00,  5.09batch/s]
Epoch 49/50, Loss: 0.0063, Train Acc: 0.9982, Val Acc: 0.5136
Epoch 49/50, Loss: 0.0063, Train Acc: 0.9982, Val Acc: 0.5136


Epoch 49/50, Loss: 0.0063, Train Acc: 0.9982, Val Acc: 0.5136


Iterating eval graphs: 100%|██████████| 65/65 [00:12<00:00,  5.02batch/s]
Epoch 50/50, Loss: 0.2539, Train Acc: 0.8496, Val Acc: 0.7023
Epoch 50/50, Loss: 0.2539, Train Acc: 0.8496, Val Acc: 0.7023


Epoch 50/50, Loss: 0.2539, Train Acc: 0.8496, Val Acc: 0.7023


In [32]:
import gc
del train_dataset
del train_loader
del full_dataset
del val_dataset
del val_loader
gc.collect()

11621

In [33]:
test_dataset = GraphDataset(args.test_path, transform=add_zeros)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False)
    

In [34]:
model.load_state_dict(torch.load(checkpoint_path))
predictions = evaluate(test_loader, model, device, calculate_accuracy=False)
save_predictions(predictions, args.test_path)

Iterating eval graphs: 100%|██████████| 71/71 [00:13<00:00,  5.45batch/s]

Predictions saved to /home/onyxia/work/DL-Hackathon/hackaton/submission/testset_D.csv


In [35]:
import tarfile
import os

def gzip_folder(folder_path, output_file):
    """
    Compresses an entire folder into a single .tar.gz file.

    Args:
        folder_path (str): Path to the folder to compress.
        output_file (str): Path to the output .gz file.
    """
    with tarfile.open(output_file, "w:gz") as tar:
        tar.add(folder_path, arcname=os.path.basename(folder_path))
    print(f"Folder '{folder_path}' has been compressed into '{output_file}'")

# Example usage
folder_path = "/home/onyxia/work/DL-Hackathon/hackaton/submission"            # Path to the folder you want to compress
output_file = "/home/onyxia/work/DL-Hackathon/hackaton/submission.gz"        # Output .gz file name
gzip_folder(folder_path, output_file)

Folder '/home/onyxia/work/DL-Hackathon/hackaton/submission' has been compressed into '/home/onyxia/work/DL-Hackathon/hackaton/submission.gz'
